In [ ]:
import numpy as np
from DCP import *

%load_ext autoreload
%autoreload 2


## PLaying With DCP Matrix

FM Forward Matrix

CM Color Matrix

AB Anology Balance 

In [ ]:
parser = DCPParser('.\data\Sony ILCE-7RM2 Adobe Standard.xml')
parser.parse_matrices()
CM1 = parser.matrices['ColorMatrix1']
CM2 = parser.matrices['ColorMatrix2']
FM1 = parser.matrices['ForwardMatrix1']
FM2 = parser.matrices['ForwardMatrix2']

CameraWB = np.array([2592.0, 1024.0, 1556.0]) 
CameraN = 1/CameraWB
# AB = np.diag([1.83203125,1.0       ,3.15234375])
# BA = np.diag(1/np.array([1.83203125,1.0       ,3.15234375]))


In [ ]:
# Color Matrix Properties
# WB Temp calculation
CM = CM2
# CM = interpolateMatrix(2870,6500,3050,CM1,CM2)

XYZ2Camera = CM # AB * CM
Camer2XYZ = np.linalg.inv(CM) # inv(CM)  * inv(AB) 
# XYZ white point = Camer2XYZ * Camera netural point
# Camera netural point is (1/Camera WB) just CameraRGB response to white/netural
XYZ_wp = Camer2XYZ @ CameraN

x,y,Y = XYZ2xyY(XYZ_wp)
print(f"Color Temp {xy2Temp(x,y),xyCoordToTemperature([x,y])}") 


In [ ]:
# Forawrd Matrix Properties
# Should be D50 in XYZ
print(FM1 @ [1,1,1]) # D50 in XYZ
print(FM2 @ [1,1,1]) # D50 in XYZ

# DCP ISP process

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ISP import *
from DCP import *

# %matplotlib notebook
%matplotlib qt5

%load_ext autoreload
%autoreload 2


In [ ]:
img,H_raw = read_raw_simple("./data/0.ARW")

path ='./data/Sony ILCE-7RM2 Adobe Standard.xml'
# path = './data/Olympus E-M1 Mark III Adobe Standard.xml'

parser = DCPParser(path)
parser.parse_matrices()
CM1 = parser.matrices['ColorMatrix1']
CM2 = parser.matrices['ColorMatrix2']
FM1 = parser.matrices['ForwardMatrix1']
FM2 = parser.matrices['ForwardMatrix2']

# Camera AWB
R_gain,G_gain,B_gain = H_raw.camera_whitebalance[0:3]
cameraWB = np.array([R_gain/G_gain,1,B_gain/G_gain])
cameraNetural = 1/cameraWB # Actuall is the RGB vector represent white

In [ ]:
# WB Temp calculation
CM = CM1
CM = interpolateMatrix(2870,6500,3050,CM1,CM2)

XYZ2Camera = CM # AB * CM
Camer2XYZ = np.linalg.inv(CM) # inv(CM)  * inv(AB) 
# XYZ white point = Camer2XYZ * Camera netural point
# Camera netural point is (1/Camera WB) just CameraRGB response
XYZ_wp = Camer2XYZ @ cameraNetural

x,y,Y = XYZ2xyY(XYZ_wp)
xy2Temp(x,y),xyCoordToTemperature([x,y])

In [ ]:
# D50 XYZ 2 sRGB D65
#https://www.russellcottrell.com/photo/matrixCalculator.htm
XYZ2RGB = np.array(
[[ 3.1342599, -1.6171978, -0.4906852],
 [-0.9787551,  1.9161350,  0.0334462],
 [ 0.0719423, -0.2289582,  1.4052060]])

XYZ2RGB @ [0.96422,	1.00000,	0.82521] # XYZ2RGB * D50XYZ should be [1,1,1]

In [ ]:
# Forward Matrix
FM = FM2
D = np.diag(cameraWB)
Camera2XYZD50 = FM @ D

vector, invf = im2vector(img) # [N,3]

wb_img = D @ vector.T # AWBed CameraRGB
XYZ_img = Camera2XYZD50 @ vector.T # [3,N] in D50
sRGB_img = XYZ2RGB @ XYZ_img # linear RGB


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plotting the first image
axs[0, 0].imshow(img)
axs[0, 0].set_title('Camera RGB (No gamma)')

# Plotting the second image
axs[0, 1].imshow(invf(gamma(wb_img.T)))
axs[0, 1].set_title('AWB Camera RGB')

# Plotting the third image
axs[1, 0].imshow(invf(XYZ_img.T))
axs[1, 0].set_title('XYZ D50')

# Plotting the fourth image
axs[1, 1].imshow(invf(gamma(sRGB_img.T)))
axs[1, 1].set_title('sRGB')

plt.show()

In [ ]:
# Test Different Colors

color1 = FM1 @ wb_img # [N,3]
color2 = FM2 @ wb_img

RGB1 = gamma(XYZ2RGB @ color1).T # linear RGB > sRGB
RGB2 = gamma(XYZ2RGB @ color2).T

RGBC = gamma(wb_img).T




In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plotting the second image
axs[0, 0].imshow(invf(RGBC))
axs[0, 0].set_title('AWB Camera RGB')

# Plotting the first image
axs[0, 1].imshow(invf(RGB2))
axs[0, 1].set_title('FM2 sRGB')

# Plotting the third image
axs[1, 0].imshow(invf(RGB1))
axs[1, 0].set_title('FM1 sRGB')

axs[1, 1].imshow(invf(RGB1-RGB2))
axs[1, 1].set_title('FM1 FM2 diff')

In [ ]:
from PIL import Image  
rgb_raw_img = Image.fromarray(np.uint8(invf(RGB1) * 255))  # 将像素值转换为 0-255 范围内的整数
rgb_raw_img.save('./data/FM_RGB.jpg')  # 保存图片 

In [ ]:
# PLay with FM

XYZ2RGB @ FM1

In [ ]:
plt.figure()
plt.imshow(invf(gamma(wb_img.T)))
plt.title("AWB Camera RGB")
print(wb_img.max(),wb_img.min())

In [ ]:
XYZ2RGB @ FM